# In this file we create an ANN model ( testing Purpose ) and fine tune this model for production and deployment.

In [ ]:
##  Import Libraries

import keras
from keras.models import Sequential 

keras.__version__

import tensorflow 
tensorflow.__version__






'2.18.0'

In [1]:
from tensorflow.keras.models import Sequential